# Road Following by Classification - ロボットの自律走行


## はじめに
いよいよ学習済みモデルを使って自律走行させてみましょう。Google Colabで学習させた`best_model_resnet18.pth`をJetBotにアップロードしてください。

## TensorRTモデルの作成
Google Colabで学習したPyTorch形式のモデル`best_model_resnet18.pth`は JetBot上では推論速度が遅いのでより高速に処理ができるTensorRT形式に変換しましょう。

In [ ]:
import torch
import torchvision
import torchvision.models as models
import torch.nn as nn

def create_resnet18():
    model = models.resnet18(pretrained=True)
    model.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(512, 3)
    )
    return model

model = create_resnet18().cuda().eval().half()

model.load_state_dict(torch.load('best_model_resnet18.pth'))

In [ ]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()
model_trt = torch2trt(model, [data], fp16_mode=True)

torch.save(model_trt.state_dict(), 'best_model_resnet18_trt.pth')

## TensorRTモデルのロード
TensorRT形式のモデル`best_model_resnet18_trt.pth`を読み込みましょう。

In [ ]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_model_resnet18_trt.pth'))

## 画像の前処理関数

入力画像に対して画像分類AIが正しく推論できるように前処理を行う関数を定義します。

In [ ]:
import cv2
import numpy as np

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = x.cuda().half()
    x = x[None, ...]
    x = x/255.
    return x

## Cameraの準備
カメラが正しく動くか確認しましょう。

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

image = widgets.Image(format='jpeg', width=300, height=300)
camera = Camera.instance(width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

## 試しに推論
きちんと推論できるか試してみましょう。

In [ ]:
import torch.nn.functional as F

with torch.no_grad():
    x = camera.value
    x = preprocess(x)
    y = model_trt(x)
    y = F.softmax(y, dim=1)
    y_idx = torch.argmax(y, dim=1).item()
    print(y_idx)

## Robotの作成
ロボットのモーターを制御するためにRobotクラスのインスタンスを作成しましょう。

In [ ]:
from jetbot import Robot

robot = Robot()

### ロボットの動作関数の定義
左右のモーターの出力値を設定してロボットを動作させるための関数を定義しましょう。

In [ ]:
def move_forward():
    robot.set_motors(0.5, 0.5)

def turn_left():
    robot.set_motors(0.2, 0.5)

def turn_right():
    robot.set_motors(0.5, 0.2)
    
actions_dict = {0:"Go Forward", 1:"Turn Left", 2:"Turn Right"}

## 自律走行
ロボットを走らせてみましょう。

In [ ]:
move_robot = True # Trueのときロボットが動く，Falseのとき動かない

import time
t0 = time.time()

display(image)
steps = 1000 # 値を大きくすると長い時間ロボットが動く

softmax = torch.nn.Softmax(dim=1)

with torch.no_grad():

    for i in range(steps):
        x = camera.value
        x = preprocess(x)
        y = model_trt(x)
        y = softmax(y)
        y_idx = torch.argmax(y, dim=1).item()
        
        if move_robot:
            if y_idx == 0: move_forward()
            elif y_idx == 1: turn_left()
            elif y_idx == 2: turn_right()
        
        now = time.time()
        dt = now-t0
        t0 = now
        FPS = 1/dt
        
        print(f"\rStep:{i+1}/{steps}   Action:{actions_dict[y_idx]}   FPS:{FPS:.0f}", end="")

robot.stop()

## 終了
終了する場合は次のセルを実行してください。

In [ ]:
robot.stop()
camera.stop()